In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [48]:
dataset = pd.read_csv('emissao_de_vistos.csv')
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003


In [40]:
resultado = dataset.loc[:, ['CASE_STATUS']].dropna()
resultado.head()

,CASE_STATUS
0,CERTIFIED-WITHDRAWN
1,CERTIFIED-WITHDRAWN
2,CERTIFIED-WITHDRAWN
3,CERTIFIED-WITHDRAWN
4,WITHDRAWN


In [46]:
profissao = dataset.loc[:, ['JOB_TITLE']].dropna()
profissao.head()

,JOB_TITLE
0,POSTDOCTORAL RESEARCH FELLOW
1,CHIEF OPERATING OFFICER
2,CHIEF PROCESS OFFICER
3,"REGIONAL PRESIDEN, AMERICAS"
4,PRESIDENT MONGOLIA AND INDIA


In [45]:
tempo = dataset.loc[:, ['FULL_TIME_POSITION']].dropna()
tempo.head()

,FULL_TIME_POSITION
0,N
1,Y
2,Y
3,Y
4,Y


In [49]:
salario = dataset.loc[:, ['PREVAILING_WAGE']].dropna()
salario.head()

,PREVAILING_WAGE
0,36067.0
1,242674.0
2,193066.0
3,220314.0
4,157518.4


In [52]:
R_P = resultado.join(profissao, how='inner')
R_P.head()

,CASE_STATUS,JOB_TITLE
0,CERTIFIED-WITHDRAWN,POSTDOCTORAL RESEARCH FELLOW
1,CERTIFIED-WITHDRAWN,CHIEF OPERATING OFFICER
2,CERTIFIED-WITHDRAWN,CHIEF PROCESS OFFICER
3,CERTIFIED-WITHDRAWN,"REGIONAL PRESIDEN, AMERICAS"
4,WITHDRAWN,PRESIDENT MONGOLIA AND INDIA


In [53]:
R_T = resultado.join(tempo, how='inner')
R_T.head()

,CASE_STATUS,FULL_TIME_POSITION
0,CERTIFIED-WITHDRAWN,N
1,CERTIFIED-WITHDRAWN,Y
2,CERTIFIED-WITHDRAWN,Y
3,CERTIFIED-WITHDRAWN,Y
4,WITHDRAWN,Y


In [55]:
R_S =resultado.join(salario, how='inner')
R_S.head()

,CASE_STATUS,PREVAILING_WAGE
0,CERTIFIED-WITHDRAWN,36067.0
1,CERTIFIED-WITHDRAWN,242674.0
2,CERTIFIED-WITHDRAWN,193066.0
3,CERTIFIED-WITHDRAWN,220314.0
4,WITHDRAWN,157518.4
